# Imports

In [1]:
import tensorflow as tf
import keras, keras.layers as L, keras.backend as K
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
from keras.models import save_model
from keras.datasets import cifar10, fashion_mnist
from keras.models import load_model
from annoy import AnnoyIndex
from sklearn.metrics import classification_report
from keras.utils import np_utils
import yagmail
import os

%matplotlib inline

/home/nicoroble/anaconda3/envs/tensorflow-gpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
email = os.environ.get('GMAIL')
pswd = os.environ.get('GMAILPASS')
yag = yagmail.SMTP(email, pswd)

In [3]:
def reset_tf_session():
    curr_session = tf.get_default_session()
    # close current session
    if curr_session is not None:
        curr_session.close()
    # reset graph
    K.clear_session()
    # create new session
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    s = tf.InteractiveSession(config=config)
    K.set_session(s)

In [4]:
reset_tf_session()

# Loading the Models

In [5]:
bit_size = 32
architecture = '1024'

## Base

In [6]:
# autoencoder_id='final-multitask09'
# model_path = F'fashion-models/bitsize{bit_size}/{architecture}architecture/'
# basefinal = load_model(F'{model_path}encoder{autoencoder_id}.h5')

In [7]:
autoencoder_id='multitaskfinal'
model_path = 'fashion-models/'
multitask = load_model(F'{model_path}encoder{autoencoder_id}.h5')

/home/nicoroble/anaconda3/envs/tensorflow-gpu/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


# Importing the Fashion MNIST Dataset

In [8]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [9]:
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=0.2, random_state=33)

In [10]:
x_test = x_test.reshape([-1, 28, 28, 1])
x_val = x_val.reshape([-1, 28, 28, 1])

IMG_SHAPE = x_test.shape[1:]

# center images
x_test = x_test.astype('float32') / 255.0
x_val = x_val.astype('float32') / 255.0

# Encoding the images

In [11]:
# base_test_codes = basefinal.predict(x_test)
# base_val_codes = basefinal.predict(x_val)

In [12]:
multitask_test_codes = multitask.predict(x_test)
multitask_val_codes = multitask.predict(x_val)

In [13]:
multitask_test_codes.shape

(2000, 32)

In [14]:
multitask_val_codes.shape

(8000, 32)

# Approximate Nearest Neighbor Search Indeces

In [15]:
# base_index = AnnoyIndex(bit_size, metric='angular')
# for i in range(len(base_val_codes)):
#     base_index.add_item(i, base_val_codes[i])

In [16]:
# base_index.build(1000)

In [17]:
multitask_index = AnnoyIndex(bit_size, metric='angular')
for i in range(len(multitask_val_codes)):
    multitask_index.add_item(i, multitask_val_codes[i])

In [18]:
multitask_index.build(1000)

True

In [19]:
def get_scores(indexer, test_codes, n):
    y_true_vals = []
    y_pred_vals = []
    
    for i in range(len(test_codes)):
        y_true = [y_test[i]] * n
        for val in y_true:
            y_true_vals.append(val)
        
        y_pred = indexer.get_nns_by_vector(vector=test_codes[i],
                                           n=n,
                                           search_k=200000
                                          )
        y_pred = y_val[y_pred]
        for val in y_pred:
            y_pred_vals.append(val)
            
    result = classification_report(y_true_vals, y_pred_vals, output_dict=True)['weighted avg']
    
    return result

In [20]:
result = get_scores(multitask_index, multitask_test_codes, 10)